In [16]:
from bs4 import BeautifulSoup
import html2text
from glob import glob
from pathlib import Path
import json

In [20]:
def clean_webpage(page):
  cleaned_page = []
  for line in page.split("\n"):
    line_text = line.strip()
    
    if not line_text:
      continue
    
    cleaned_line = " ".join(line_text.split())
    cleaned_page.append(cleaned_line)
  return "\n".join(cleaned_page)
  

In [33]:
def get_question_answer(html_path):
  h = html2text.HTML2Text()
  h.ignore_images = True
  h.tag_stack = False
  h.drop_white_space = True
  
  html_path = Path(html_path)
  html_file = open(html_path)
  soup = BeautifulSoup(html_file)
  html_file.close()
  
  question = soup.find(id="kb_article_question")
  if not question:
    return None
  question = question.text.strip()
  
  answer = soup.find(id="kb_article_text")
  if not answer:
    return None
  answer = answer.prettify()
  
  return {
    "id": html_path.stem,
    "question": question,
    "answer": clean_webpage(h.handle(answer))
  } 

In [34]:
data = []

for path in glob("../html-archive/*.html"):
  question_answer = get_question_answer(path)
  
  if not question_answer:
    continue
    
  data.append(question_answer)

with open("../data/html-data.json", "w+") as html_data_file:
  json.dump({"data": data}, html_data_file)